# Title

## Setup

In [ ]:
import sys

sys.path.append("../working")

In [ ]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [ ]:
import glob
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
from omegaconf import OmegaConf
from progressbar import progressbar
from src.get_score import get_score
from src.load_data import LoadData, PostprocessData, PreprocessData
from src.utils import df_stats

# from src.make_dataset import BaseDataset, get_transforms
# from src.make_model import ImageBaseModel
# from torch.utils.data import DataLoader

# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_colwidth", None)

In [ ]:
# Competition specific library
import math

import scanpy as sc
import scipy.stats as stats
import umap
from anndata import AnnData
from ivis import Ivis
from sklearn.preprocessing import StandardScaler

In [ ]:
c = OmegaConf.load("../working/config/main.yaml")

In [ ]:
input = PreprocessData(c, do_preprocess=False)

In [ ]:
input = LoadData(c, do_preprocess=False, use_fold=True)

In [ ]:
input = PostprocessData(c)

In [ ]:
dir(input)

## Basic Info

In [ ]:
df = ...

In [ ]:
df.info()

In [ ]:
df_stats(df)

In [ ]:
df.head()

In [ ]:
adata = AnnData(input.train_cite_inputs.to_numpy())
adata.obs_names = input.train_cite_inputs.index
adata.var_names = input.train_cite_inputs.columns
adata

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)

sc.pp.filter_genes(adata, min_cells=300)

In [ ]:
adata.var["mt"] = adata.var_names.str.contains("_MT-")

sc.pp.calculate_qc_metrics(
    adata,
    qc_vars=["mt"],
    percent_top=None,
    log1p=False,
    inplace=True,
)

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
sc.pp.highly_variable_genes(adata)
adata = adata[:, adata.var["highly_variable"]]

In [ ]:
sc.pp.scale(adata, max_value=10)

sc.tl.pca(adata, svd_solver="arpack")

In [ ]:
X = adata.obsm["X_pca"]

In [ ]:
ivis = Ivis(k=15, model="maaten", n_epochs_without_progress=5)
ivis.fit(X)
embeddings = ivis.transform(X)

In [ ]:
adata.var.info()

In [ ]:
[col for col in adata.var_names if "CST" in col]

In [ ]:
fill = adata.X[:, adata.var.index == "ENSG00000101439_CST3"]
fill = fill.reshape((X.shape[0],))

In [ ]:
fill.shape

In [ ]:
adata.obs.info()

In [ ]:
embeddings.shape

In [ ]:
plt.figure(figsize=(6, 4), dpi=150)
sc = plt.scatter(x=embeddings[:, 0], y=embeddings[:, 1], c=fill, s=5)
plt.xlabel("ivis 1")
plt.ylabel("ivis 2")
plt.title("CST3")
plt.colorbar(sc)